In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
########################################################################
#
# Copyright (c) 2017 4paradigm.com, Inc. All Rights Reserved
#
########################################################################

"""
File: data_analysis.py
Author: wangyusen(wangyusen@4paradigm.com)
"""

import numpy as np
import pandas as pd
import os
import datetime
import time
import warnings
warnings.filterwarnings('ignore')
# from matplotlib.pylab import rcParams
# from matplotlib.font_manager import *
# import matplotlib.pyplot as plt
# %matplotlib inline
# #定义自定义字体，文件名从fc-list :lang=zh查看系统中文字体中来  
# myfont = FontProperties(fname='/System/Library/Fonts/PingFang.ttc')  
#解决负号'-'显示为方块的问题  
# rcParams['axes.unicode_minus']=False  
# mpl.rcParams['font.sans-serif'] = ['SimHei']
# mpl.rcParams['font.serif'] = ['SimHei']
# import seaborn as sns
# sns.set_style("darkgrid",{"font.sans-serif":['simhei', 'Arial']})

In [2]:
import numpy as np
import pandas as pd
import os
import datetime
import sys
import re
import json
import time
import zipfile
# import cpca  # needs manually installing
from sklearn import preprocessing


unique = lambda x:len(set(x))  # 唯一值的个数

def read_file(f,**kw):
    '''读文件'''
    try:
        tmp = pd.read_csv(f,**kw)
    except:
        try:
            tmp = pd.read_excel(f,**kw)
        except:
            try:
                tmp = pd.read_table(f,**kw)
            except:
                print('Wrong file: ',f)
                return None
    return tmp

def walk_zip_files(path,pattern_zip='.*.zip',pattern_file='.*trx.*.csv',colnum=False,
                   pattern_date='201[89].\d{2}.\d{2}',**kw):
    '''
    解析文件夹下所有zip里的符合pattern的文件
    参数：
    -- path：文件路径
    -- pattern_zip：zip文件正则表达式
    -- pattern_file：文本文件正则表达式
    -- colnum：文件列数；int；默认False代表不用指定列数来过滤文件
    -- pattern_date：文件日期正则表达式，False表示不取日期列
    -- **kw：文件读取参数
    返回：
    -- 符合条件的所有文件合并后的dataframe
    '''
    import zipfile
    import re
    trx = None

    for dirpath,dirnames,filenames in os.walk(path):
        for file in filenames:
            if re.search(pattern_zip,file):
                fullpath=os.path.join(dirpath,file)
                with zipfile.ZipFile(fullpath, "r") as z:
                    for i in z.namelist():
                        if re.search(pattern_file,i):
                            f = z.open(i)
#                             print('reading: ',fullpath)
                            tmp = read_file(f,**kw)
                            try:
                                print(fullpath,i,'行列数：',tmp.shape)
                                if pattern_date:
                                    try:
                                        tmp['file_date'] = re.findall(string=fullpath,pattern=pattern_date)[0]
                                    except: pass
                                if colnum:
                                    if tmp.shape[1] == colnum:
                                        trx = pd.concat([trx,tmp],axis=0,ignore_index=True)
                                else:
                                    trx = pd.concat([trx,tmp],axis=0,ignore_index=True)
                            except: pass           
    return trx

def walk_files(path,pattern_file='.*loan.*csv',colnum=False,pattern_date='201[89].\d{2}.\d{2}',**kw):
    '''
    提取指定path目录下所有符合pattern正则式文件的内容
    参数：
    -- path：文件路径
    -- pattern_file：文件名正则表达式
    -- colnum：列数;int；默认False代表不用指定列数来过滤文件
    -- pattern_date：文件日期正则表达式，从文本文件路径中得到
    -- **kw：文件读取参数
    返回：
    -- 符合条件的所有文件合并后的dataframe
    '''
    import re
    data = None
    for dirpath,dirnames,filenames in os.walk(path):
        for file in filenames:
            if re.search(pattern_file,file):
                fullpath=os.path.join(dirpath,file)
#                 print('reading: ',fullpath)
                tmp = read_file(fullpath,**kw)
                try:
                    print(fullpath,'行列数：',tmp.shape)
                    if pattern_date:
                        ## 一般路径或文件名会含有文件上传时间，匹配pattern_date正则来解析文件日期
                        try:
                            tmp['file_date'] = re.findall(string=fullpath,pattern=pattern_date)[0]
                        except:pass
                    if colnum:
                        if tmp.shape[1] == colnum:
                            data = pd.concat([data,tmp],axis=0,ignore_index=True)
                    else:
                        data = pd.concat([data,tmp],axis=0,ignore_index=True)
                except: pass
    return data



def convert_time(time):
    '''字符转换为时间'''
    if not time:
        return
    try:
        return datetime.datetime.strptime(str(time), "%Y-%m-%d")
    except:
        try:
            return datetime.datetime.strptime(str(time), "%Y-%m-%d %H:%M:%S")
        except:
            try:
                return datetime.datetime.strptime(str(time), "%Y%m%d%H%M%S")
            except:
                try:
                    return datetime.datetime.strptime(str(time), "%Y%m%d")
                except:
                    try:
                         return datetime.datetime.strptime(str(time), "%Y/%m/%d")
                    except:
                        try:
                            return datetime.datetime.strptime(str(time), "%Y/%m/%d %H:%M")
                        except:
                            try:
                                return datetime.datetime.strptime(str(time), "%Y/%m/%d %H:%M:%S")
                            except:
                                try:
                                    return datetime.datetime.strptime(str(time), "%Y-%m-%d %H:%M")
                                except:
                                    try:
                                        return datetime.datetime.strptime(str(time), "%Y-%m-%dT%H:%M:%S.000+08:00")
                                    except:
                                        print("Wrong date format : %s " % time)
                                        return None

def convert_time_col(df,pattern_date='(time|date|ddl)',suffix=''):
    '''
    对df含有时间类型的列，其字符串转换成标准时间
    参数：
    -- df:dataframe
    -- pattern_date:时间列表达式
    -- suffix：时间列添加后缀
    '''
    columns = df.columns
    # 时间类型
    time_cols = columns[columns.str.lower().str.contains(pat=pattern_date)]
    for time_col in time_cols:
        df[str(time_col)+suffix] = df[time_col].apply(convert_time)   
    return df

In [3]:
# print(os.listdir())
# excel_path = './建模基础字段及客户交易数据.xlsx'
# excel_path2 = './客户订单详情.xlsx'

ORDER = pd.read_excel('C:/Users/Vicky/Desktop/范式/中驰进件数据-V2-2020026.xlsx')


In [ ]:
#BUYER = pd.read_excel('C:/Users/Vicky/Desktop/范式/建模基础字段及客户交易数据.xlsx', sheet_name=2, encoding='utf-8')

In [ ]:
#LOAN = pd.read_excel('C:/Users/Vicky/Desktop/范式/建模基础字段及客户交易数据.xlsx', sheet_name=4, encoding='utf-8')

In [4]:
ORDER

,订单编号,订单头编号,客户,客户类型,卖家,卖家类型,收货地址,收货人,收货人电话,收货地邮编,...,下单数量,优惠金额,优惠劵抵扣金额,应付金额,下单时间,发货时间,发货数量,收货时间,收货数量,订单状态
0,13940938001,13940938,913859,NaN,6102,NaN,北京 ****西台下村,韩维霞,137****2377,NaN,...,1,NaN,NaN,38.00,2018-01-02,2018-01-02,1.0,2018-01-02,1.0,已完成
1,13946858006,13946858,910427,NaN,6068,NaN,河南 ****力帆售后,庄书兵,159****8531,NaN,...,5,NaN,NaN,77.50,2018-01-01,2018-01-02,5.0,2018-01-02,5.0,已完成
2,13948862005,13948862,960318,NaN,35,NaN,河南 ****航路中段,杨志红,137****0805,NaN,...,30,NaN,NaN,3000.00,2018-01-02,2018-01-02,30.0,2018-01-02,30.0,已完成
3,13953023006,13953023,933376,NaN,35,NaN,河南 ****顺达汽修,王腾飞,185****2991,NaN,...,5,NaN,NaN,105.00,2018-01-02,2018-01-02,5.0,2018-01-02,5.0,已完成
4,13957873002,13957873,QXC3643278243,NaN,6008,NaN,北京 ****油站内）,杜守朝,159****8735,NaN,...,3,NaN,NaN,120.00,2018-01-01,2018-01-03,3.0,2018-01-03,3.0,已完成
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706747,21896995005,21896995,QXC4472103658,NaN,GYS648006112,NaN,山东 ****有限公司,泰安开发区四通运输有,131****2236,NaN,...,15,NaN,NaN,5460.00,2020-04-24,2020-04-24,15.0,2020-04-24,NaN,已发货
706748,21897004008,21897004,QXC4478112163,NaN,GYS648006112,NaN,山东 ****北街5号,影山交通,053****8916,NaN,...,23,NaN,NaN,10557.00,2020-04-24,2020-04-24,23.0,2020-04-24,NaN,已发货
706749,21897075002,21897075,JXS858858913,NaN,GYS576935260,NaN,北京 ****路16号,冯光吉,010****3359,NaN,...,14,NaN,NaN,9305.10,2020-04-25,2020-04-25,14.0,2020-04-25,NaN,已发货
706750,21902829016,21902829,JXS570571126,NaN,JXS552560712,NaN,北京 ****村工业园,刘宏宇,183****6660,NaN,...,37,NaN,NaN,3293.00,2020-04-21,2020-04-21,37.0,2020-04-22,37.0,已完成


In [5]:
ORDER.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 706752 entries, 0 to 706751
Data columns (total 24 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   订单编号     706752 non-null  int64         
 1   订单头编号    706752 non-null  int64         
 2   客户       706752 non-null  object        
 3   客户类型     0 non-null       float64       
 4   卖家       706752 non-null  object        
 5   卖家类型     0 non-null       float64       
 6   收货地址     706752 non-null  object        
 7   收货人      706752 non-null  object        
 8   收货人电话    706744 non-null  object        
 9   收货地邮编    0 non-null       float64       
 10  商品       706752 non-null  object        
 11  品牌       677369 non-null  object        
 12  品类       628653 non-null  object        
 13  单价       706752 non-null  float64       
 14  下单数量     706752 non-null  int64         
 15  优惠金额     0 non-null       float64       
 16  优惠劵抵扣金额  0 non-null       float64       
 17  应付金额     7

In [ ]:
#################################################
#基本信息
#################################################

In [6]:
#buyer = BUYER[[u'企业名称', u'类型', u'经营地址\n（非必填）', u'企业证照类型', u'统一社会信用代码证', u'状态',u'会员注册时间']]
               
#buyer.columns = ['company_name','company_type','company_address','company_ID_type','company_ID_num','company_status','company_redister_time']
                 

order = ORDER[[u'订单头编号', u'客户', u'品类', u'单价', u'下单数量', u'下单时间', u'优惠金额', u'优惠劵抵扣金额', u'发货时间', u'发货数量', u'收货时间', u'收货数量', u'订单状态',u'应付金额']]
      
order.columns = ['order_ID','company_name','type_of_merchandize', 'unit_price', 'order_num','order_time', 'discount_amt','expense_deductible','send_time','send_num','receive_time','receive_num','order_status', 'pay_amt']
                
               

#loan = LOAN[[u'贷款主体', u'订单头编号', u'贷款金额', u'生成时间', u'账单日', u'截止还款日',  u'贷款状态', u'逾期利率', u'逾期利息', u'放款时间', u'放款金额', u'还款金额', u'关闭时间']]
            
#loan.columns = ['company_name','order_ID','request_amt','request_time','bill_time','return_time', 'loan_status','overdue_rate','overdue_interest','loan_time','loan_amt','return_amt','close_time']
              

In [7]:
order

,order_ID,company_name,type_of_merchandize,unit_price,order_num,order_time,discount_amt,expense_deductible,send_time,send_num,receive_time,receive_num,order_status,pay_amt
0,13940938,913859,空调滤清器,38.00,1,2018-01-02,NaN,NaN,2018-01-02,1.0,2018-01-02,1.0,已完成,38.00
1,13946858,910427,燃油滤清器,15.50,5,2018-01-01,NaN,NaN,2018-01-02,5.0,2018-01-02,5.0,已完成,77.50
2,13948862,960318,汽机油,100.00,30,2018-01-02,NaN,NaN,2018-01-02,30.0,2018-01-02,30.0,已完成,3000.00
3,13953023,933376,空气滤清器,21.00,5,2018-01-02,NaN,NaN,2018-01-02,5.0,2018-01-02,5.0,已完成,105.00
4,13957873,QXC3643278243,机油滤清器,40.00,3,2018-01-01,NaN,NaN,2018-01-03,3.0,2018-01-03,3.0,已完成,120.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706747,21896995,QXC4472103658,后轮轮胎,364.00,15,2020-04-24,NaN,NaN,2020-04-24,15.0,2020-04-24,NaN,已发货,5460.00
706748,21897004,QXC4478112163,后轮轮胎,459.00,23,2020-04-24,NaN,NaN,2020-04-24,23.0,2020-04-24,NaN,已发货,10557.00
706749,21897075,JXS858858913,轮胎,664.65,14,2020-04-25,NaN,NaN,2020-04-25,14.0,2020-04-25,NaN,已发货,9305.10
706750,21902829,JXS570571126,前刹车片,89.00,37,2020-04-21,NaN,NaN,2020-04-21,37.0,2020-04-22,37.0,已完成,3293.00


In [38]:
def convert_time(time):
    try:
        return datetime.datetime.strptime(time, "%Y-%m-%d %H:%M:%S")
    except:
        return ''

#buyer['company_redister_time'] = buyer['company_redister_time'].apply(lambda x: convert_time(x))

order['order_time'] = order['order_time'].apply(lambda x: convert_time(x))
order['send_time'] = order['send_time'].apply(lambda x: convert_time(x))
order['receive_time'] = order['receive_time'].apply(lambda x: convert_time(x))

#loan['request_time'] = loan['request_time'].apply(lambda x: convert_time(x))
#loan['bill_time'] = loan['bill_time'].apply(lambda x: convert_time(x))
#loan['return_time'] = loan['return_time'].apply(lambda x: convert_time(x))
#loan['loan_time'] = loan['loan_time'].apply(lambda x: convert_time(x))
#loan['close_time'] = loan['close_time'].apply(lambda x: convert_time(x))

In [8]:
order=convert_time_col(order)

Wrong date format : NaT 
Wrong date format : NaT 
Wrong date format : NaT 
Wrong date format : NaT 
Wrong date format : NaT 


In [9]:
order

,order_ID,company_name,type_of_merchandize,unit_price,order_num,order_time,discount_amt,expense_deductible,send_time,send_num,receive_time,receive_num,order_status,pay_amt
0,13940938,913859,空调滤清器,38.00,1,2018-01-02,NaN,NaN,2018-01-02,1.0,2018-01-02,1.0,已完成,38.00
1,13946858,910427,燃油滤清器,15.50,5,2018-01-01,NaN,NaN,2018-01-02,5.0,2018-01-02,5.0,已完成,77.50
2,13948862,960318,汽机油,100.00,30,2018-01-02,NaN,NaN,2018-01-02,30.0,2018-01-02,30.0,已完成,3000.00
3,13953023,933376,空气滤清器,21.00,5,2018-01-02,NaN,NaN,2018-01-02,5.0,2018-01-02,5.0,已完成,105.00
4,13957873,QXC3643278243,机油滤清器,40.00,3,2018-01-01,NaN,NaN,2018-01-03,3.0,2018-01-03,3.0,已完成,120.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706747,21896995,QXC4472103658,后轮轮胎,364.00,15,2020-04-24,NaN,NaN,2020-04-24,15.0,2020-04-24,NaN,已发货,5460.00
706748,21897004,QXC4478112163,后轮轮胎,459.00,23,2020-04-24,NaN,NaN,2020-04-24,23.0,2020-04-24,NaN,已发货,10557.00
706749,21897075,JXS858858913,轮胎,664.65,14,2020-04-25,NaN,NaN,2020-04-25,14.0,2020-04-25,NaN,已发货,9305.10
706750,21902829,JXS570571126,前刹车片,89.00,37,2020-04-21,NaN,NaN,2020-04-21,37.0,2020-04-22,37.0,已完成,3293.00


In [10]:
def deal_name(x):
    try:
        return x.strip().strip('-P')
    except:
        return str(x).strip().strip('-P')

def del_test(x):
    try:
        return u'测试' not in x
    except:
        return u'测试' not in str(x)

def deal_id(x):
    if x[0]=="'":
        return x[1:]
    else:
        return x

order = order[order['company_name'].apply(lambda x: del_test(x))]

order['company_name'] = order['company_name'].apply(lambda x: deal_name(x))
#order[u'客户id'] = order[u'客户id'].apply(lambda x: deal_id(x))

In [21]:
# 最早交易时间
tmp = order.groupby(['company_name'])['order_time'].min().to_frame()
tmp.columns=[u'最早交易时间']
result = tmp.reset_index()
result.columns = [u'企业名称',u'最早交易时间']
# result = pd.merge(buyer[['company_name']], tmp, left_on=[u'企业名称'], right_index=True, how='right')

# 最晚交易时间
tmp = order.groupby(['company_name'])['order_time'].max().to_frame()
tmp.columns=[u'最近交易时间']
result = pd.merge(result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 下单总次数
tmp = order.groupby(['company_name'])['order_status'].count().to_frame()
tmp.columns=[u'下单总次数']
result = pd.merge(result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 取消总次数
tmp = order[order['order_status']==u'其他'].groupby(['company_name'])['order_status'].count().to_frame()
tmp.columns=[u'取消总次数']
result = pd.merge(result, tmp, left_on=[u'企业名称'], right_index=True, how='left')


order_finish = order[order['order_status']==u'已完成']
# 完成总次数
tmp = order_finish.groupby(['company_name'])['order_status'].count().to_frame()
tmp.columns=[u'完成总次数']
result = pd.merge(result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 完成率
result=result.fillna(0)
result[u'完成率'] = result[u'完成总次数'] / result[u'下单总次数'].apply(lambda x:float(x))
result[u'完成率'] = result[u'完成率'].apply(lambda x: round(x,4))

# 平均交易周期
result[u'平均交易周期（天）'] = (result[u'最近交易时间']-result[u'最早交易时间']) / result[u'下单总次数']
result[u'平均交易周期（天）'] = result[u'平均交易周期（天）'].apply(lambda x: round(x.total_seconds()/3600/24,1))

# 完成交易总金额
tmp = order_finish.groupby(['company_name'])['pay_amt'].sum().to_frame()
tmp.columns=[u'完成交易总金额']
result = pd.merge(result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 优惠次数
tmp = order_finish[order_finish['discount_amt']>0].groupby(['company_name'])['discount_amt'].count().to_frame()
tmp.columns=[u'优惠次数']
result = pd.merge(result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 优惠金额
tmp = order_finish[order_finish['discount_amt']>0].groupby(['company_name'])['discount_amt'].sum().to_frame()
tmp.columns=['优惠总金额']
result = pd.merge(result, tmp, left_on=[u'企业名称'], right_index=True, how='left')


order_finish[u'发货缺口'] = order_finish['order_num']-order_finish['send_num']
order_finish[u'收货缺口'] = order_finish['send_num']-order_finish['receive_num']
order_finish[u'发货缺口金额'] = order_finish[u'发货缺口']*order_finish['unit_price']
order_finish[u'收货缺口金额'] = order_finish[u'收货缺口']*order_finish['unit_price']
# 发货数量少于下单数量的次数
tmp = order_finish[order_finish[u'发货缺口']>0].groupby(['company_name'])['order_ID'].count().to_frame()
tmp.columns=[u'发货缺口次数']
result = pd.merge(result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 收货数量少于发货数量的次数
tmp = order_finish[order_finish[u'收货缺口']>0].groupby(['company_name'])['order_ID'].count().to_frame()
tmp.columns=[u'收货缺口次数']
result = pd.merge(result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 发货缺口金额
tmp = order_finish.groupby(['company_name'])[u'发货缺口金额'].sum().to_frame()
tmp.columns=[u'发货缺口金额']
result = pd.merge(result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 收货缺口金额
tmp = order_finish.groupby(['company_name'])[u'收货缺口金额'].sum().to_frame()
tmp.columns=[u'收货缺口金额']
result = pd.merge(result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 品类个数
tmp = order_finish.groupby(['company_name','type_of_merchandize'])['order_ID'].count().to_frame()
tmp = tmp.reset_index().groupby(['company_name'])['type_of_merchandize'].count().to_frame()
tmp.columns=[u'品类个数']
result = pd.merge(result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 周交易密度
order_finish.index = order_finish['order_time']
tmp = order_finish.groupby(['company_name',order_finish.index.year, order_finish.index.week])['pay_amt'].count().to_frame().reset_index(level='company_name')
tmp = tmp.groupby(['company_name'])['pay_amt'].mean().to_frame()
tmp.columns=[u'周交易密度']
tmp = tmp.round(0)
result = pd.merge(result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 周交易金额
tmp = order_finish.groupby(['company_name',order_finish.index.year, order_finish.index.week])['pay_amt'].sum().to_frame().reset_index(level='company_name')
tmp = tmp.groupby(['company_name'])['pay_amt'].mean().to_frame()
tmp.columns=[u'周交易金额']
tmp = tmp.round(2)
result = pd.merge(result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 月交易密度
tmp = order_finish.groupby(['company_name',order_finish.index.year, order_finish.index.month])['pay_amt'].count().to_frame().reset_index(level='company_name')
tmp = tmp.groupby(['company_name'])['pay_amt'].mean().to_frame()
tmp.columns=[u'月交易密度']
tmp = tmp.round(0)
result = pd.merge(result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 月交易金额
tmp = order_finish.groupby(['company_name',order_finish.index.year, order_finish.index.month])['pay_amt'].sum().to_frame().reset_index(level='company_name')
tmp = tmp.groupby(['company_name'])['pay_amt'].mean().to_frame()
tmp.columns=[u'月交易金额']
tmp = tmp.round(2)
result = pd.merge(result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

order_finish = order_finish.reset_index(drop=True)
    # 过去1、2、3、6月的交易次数和交易金额和
data_time = order_finish["order_time"].max()
timediff_list = [30,60,90,180]
for timediff in timediff_list:
    timespan = datetime.timedelta(days=timediff)
    order_finish_timespan = order_finish[order_finish["order_time"] >= data_time-timespan]
    tmp = order_finish_timespan.groupby(["company_name"])["pay_amt"].agg(["count","sum"])
    tmp.columns=[u"过去{}天交易次数".format(timediff), u"过去{}天交易金额".format(timediff)]
    result = pd.merge(result, tmp, left_on=["企业名称"], right_index=True, how="left")
    
    result=result.fillna(0)


# fill NAN with 0.0
result=result.fillna(0)

order_finish_unique = order_finish[["company_name","order_ID","order_time"]].drop_duplicates()
    
def timedelta2days(tf):
    try:
        return round(tf.total_seconds()/3600/24, 2)
    except AttributeError:
        return None
        
order_finish_unique["diff"] = order_finish_unique.sort_values(by=["order_time"]).groupby(["company_name"])["order_time"].diff().apply(lambda x: timedelta2days(x))
tmp = order_finish_unique.groupby(["company_name"])["diff"].agg(["mean","std","max"])
tmp.columns=[u"交易间隔时间的均值（天）", u"交易间隔时间的标准差（天）", u"最大交易间隔时间（天）"]
#tmp = tmp.apply(lambda x: round(x,1))
result = pd.merge(result, tmp, left_on=["企业名称"], right_index=True, how="left")

    
result

,企业名称,最早交易时间,最近交易时间,下单总次数,取消总次数,完成总次数,完成率,平均交易周期（天）,完成交易总金额,优惠次数,...,过去30天交易金额,过去60天交易次数,过去60天交易金额,过去90天交易次数,过去90天交易金额,过去180天交易次数,过去180天交易金额,交易间隔时间的均值（天）,交易间隔时间的标准差（天）,最大交易间隔时间（天）
0,3003294,2019-01-21,2019-02-19,5,0.0,5.0,1.0000,5.8,9460.00,0.0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,14.500000,19.091883,28.0
1,3003295,2020-01-09,2020-02-03,2,0.0,2.0,1.0000,12.5,2124.80,0.0,...,0.00,0.0,0.00,1.0,2000.00,2.0,2124.80,25.000000,NaN,25.0
2,3005833,2018-03-14,2018-03-14,4,0.0,4.0,1.0000,0.0,0.00,0.0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.000000,NaN,0.0
3,3005889,2018-05-17,2018-05-17,1,0.0,1.0,1.0000,0.0,150700.00,0.0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,NaN,NaN,NaN
4,3006282,2018-01-14,2018-09-28,14,0.0,14.0,1.0000,18.4,41574.24,0.0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,23.363636,19.566669,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4930,QYYG00127127470,2019-06-05,2019-06-24,10,0.0,7.0,0.7000,1.9,663.00,0.0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,9.500000,13.435029,19.0
4931,QYYG00154154408,2019-08-14,2020-04-22,2149,0.0,2149.0,1.0000,0.1,3339392.29,0.0,...,423679.48,265.0,739311.08,265.0,739311.08,1565.0,2226977.14,0.785047,4.472893,66.0
4932,QYYG00162162598,2019-08-23,2020-04-20,1892,0.0,1341.0,0.7088,0.1,902779.00,0.0,...,96771.00,708.0,438223.00,708.0,438223.00,893.0,560945.00,0.987448,4.938684,57.0
4933,QYYG00177177712,2020-04-07,2020-04-23,247,0.0,247.0,1.0000,0.1,1074892.34,0.0,...,1074892.34,247.0,1074892.34,247.0,1074892.34,247.0,1074892.34,0.941176,1.675955,5.0


In [22]:
def col_desc(col):
    if col.name==u'企业名称' or col.name==u'品类':
        return u'行业平均水平'
    elif u'最早' in col.name:
        return col.min()
    elif u'最近' in col.name:
        return col.max()
    else:
        return col.mean().round(4)


In [23]:
result.loc['Row_sum'] = result.apply(lambda x: col_desc(x))

In [52]:
#out = pd.merge(result, buyer, left_on=[u'企业名称'],right_on=['company_name'],how='left')

NameError: name 'buyer' is not defined

In [24]:
today = datetime.date.today().strftime("%Y%m%d")

# out.to_csv('./Company_Data_Analysis_{0}.csv'.format(today), index=None, encoding='gbk')

In [26]:
result.to_excel('C:/Users/Vicky/Desktop/范式/中驰车福订单分析表_new.xlsx')

In [10]:
#################################################
#金融
#################################################

In [11]:
# 最早贷款时间
tmp = loan.groupby(['company_name'])['request_time'].min().to_frame()
tmp.columns=[u'最早贷款时间']
l_result = tmp.reset_index()
l_result.columns = [u'企业名称',u'最早贷款时间']
# l_result = pd.merge(buyer[[u'企业名称']], tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 最近贷款时间
tmp = loan.groupby(['company_name'])['request_time'].max().to_frame()
tmp.columns=[u'最近贷款时间']
l_result = pd.merge(l_result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 申请总次数
tmp = loan.groupby(['company_name'])['loan_status'].count().to_frame()
tmp.columns=[u'申请总次数']
l_result = pd.merge(l_result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 取消总次数
tmp = loan[loan['loan_status']==u'已取消'].groupby(['company_name'])['loan_status'].count().to_frame()
tmp.columns=[u'取消总次数']
l_result = pd.merge(l_result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 终止总次数
tmp = loan[loan['loan_status']==u'已终止'].groupby(['company_name'])['loan_status'].count().to_frame()
tmp.columns=[u'终止总次数']
l_result = pd.merge(l_result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 完成总次数
tmp = loan[loan['loan_status']==u'已结清'].groupby(['company_name'])['loan_status'].count().to_frame()
tmp.columns=[u'完成总次数']
l_result = pd.merge(l_result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 仍活跃总次数
tmp = loan[(loan['loan_status']==u'贷款期') | (loan['loan_status']==u'待放款')].groupby(['company_name'])['loan_status'].count().to_frame()
tmp.columns=[u'仍活跃总次数']
l_result = pd.merge(l_result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 申请贷款完成率 = (完成总次数+仍活跃总次数)/申请总次数
l_result=l_result.fillna(0)
l_result[u'申请贷款完成率'] = l_result[u'完成总次数'] / (l_result[u'完成总次数']+l_result[u'终止总次数']+l_result[u'取消总次数'])
l_result[u'申请贷款完成率'] = l_result[u'申请贷款完成率'].apply(lambda x: round(x,4))


loan_finish = loan[(loan['loan_status']==u'已结清') | (loan['loan_status']==u'逾期')]
# 贷款总金额
tmp = loan_finish.groupby(['company_name'])['loan_amt'].sum().to_frame()
tmp.columns=[u'贷款总金额']
l_result = pd.merge(l_result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 平均逾期利率
tmp = loan_finish.groupby(['company_name'])['overdue_rate'].mean().to_frame()
tmp.columns=[u'平均逾期利率']
l_result = pd.merge(l_result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 逾期利息总额
tmp = loan_finish.groupby(['company_name'])['overdue_interest'].sum().to_frame()
tmp.columns=[u'逾期利息总额']
l_result = pd.merge(l_result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 逾期次数
tmp = loan_finish[loan_finish['overdue_interest']>0].groupby(['company_name'])['overdue_interest'].count().to_frame()
tmp.columns=[u'逾期次数']
l_result = pd.merge(l_result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 逾期率
l_result=l_result.fillna(0)
l_result[u'逾期率'] = l_result[u'逾期次数'] / l_result[u'完成总次数']
l_result[u'逾期率'] = l_result[u'逾期率'].apply(lambda x: round(x,4))


# loan_finish[u'放款时间'] = loan_finish[u'放款时间'].apply(lambda x: datetime.datetime.strptime(str(x), "%m/%d/%Y %H:%M:%S"))
loan_finish.index = loan_finish['loan_time']
# 周贷款次数
tmp = loan_finish.groupby(['company_name',loan_finish.index.year, loan_finish.index.week])['loan_amt'].count().to_frame().reset_index(level='company_name')
tmp = tmp.groupby(['company_name'])['loan_amt'].mean().to_frame()
tmp.columns=[u'周贷款次数']
tmp = tmp.round(0)
l_result = pd.merge(l_result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 周贷款金额
tmp = loan_finish.groupby(['company_name',loan_finish.index.year, loan_finish.index.week])['loan_amt'].sum().to_frame().reset_index(level='company_name')
tmp = tmp.groupby(['company_name'])['loan_amt'].mean().to_frame()
tmp.columns=[u'周贷款金额']
tmp = tmp.round(2)
l_result = pd.merge(l_result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 月贷款次数
tmp = loan_finish.groupby(['company_name',loan_finish.index.year, loan_finish.index.month])['loan_amt'].count().to_frame().reset_index(level='company_name')
tmp = tmp.groupby(['company_name'])['loan_amt'].mean().to_frame()
tmp.columns=[u'月贷款次数']
tmp = tmp.round(0)
l_result = pd.merge(l_result, tmp, left_on=[u'企业名称'], right_index=True, how='left')

# 月贷款金额
tmp = loan_finish.groupby(['company_name',loan_finish.index.year, loan_finish.index.month])['loan_amt'].sum().to_frame().reset_index(level='company_name')
tmp = tmp.groupby(['company_name'])['loan_amt'].mean().to_frame()
tmp.columns=[u'月贷款金额']
tmp = tmp.round(2)
l_result = pd.merge(l_result, tmp, left_on=[u'企业名称'], right_index=True, how='left')



# fill NAN with 0.0
l_result=l_result.fillna(0)
l_result

,企业名称,最早贷款时间,最近贷款时间,申请总次数,取消总次数,终止总次数,完成总次数,仍活跃总次数,申请贷款完成率,贷款总金额,平均逾期利率,逾期利息总额,逾期次数,逾期率,周贷款次数,周贷款金额,月贷款次数,月贷款金额
0,南宁市新谊进口汽车维修中心有限公司,2016-11-04 15:43:08,2018-02-28 18:10:39,592,6.0,21,551,0.0,0.9533,902565.03,0.064717,0.000,0.0,0.0000,9.0,14326.43,34.0,56410.31
1,广州市红友汽车零配件有限公司,2017-08-22 17:18:56,2018-01-24 15:22:06,39,0.0,1,26,0.0,0.9630,302530.30,0.065000,32.526,1.0,0.0385,2.0,21609.31,4.0,50421.72
2,杭州朗恩汽车用品有限公司,2017-07-18 10:10:51,2018-01-16 14:43:57,58,0.0,9,49,0.0,0.8448,2356080.22,0.065000,0.000,0.0,0.0000,4.0,196340.02,8.0,392680.04


In [12]:
l_result.loc['Row_sum'] = l_result.apply(lambda x: col_desc(x))

today = datetime.date.today().strftime("%Y%m%d")
# l_result.to_csv('Loan_Data_Analysis_{0}.csv'.format(today), index=None, encoding='gbk')

In [13]:
#################################################
#品类
#################################################

In [28]:
order_finish

,order_ID,company_name,type_of_merchandize,unit_price,order_num,order_time,discount_amt,expense_deductible,send_time,send_num,receive_time,receive_num,order_status,pay_amt,发货缺口,收货缺口,发货缺口金额,收货缺口金额
0,13940938,913859,空调滤清器,38.00,1,2018-01-02,NaN,NaN,2018-01-02,1.0,2018-01-02,1.0,已完成,38.00,0.0,0.0,0.0,0.0
1,13946858,910427,燃油滤清器,15.50,5,2018-01-01,NaN,NaN,2018-01-02,5.0,2018-01-02,5.0,已完成,77.50,0.0,0.0,0.0,0.0
2,13948862,960318,汽机油,100.00,30,2018-01-02,NaN,NaN,2018-01-02,30.0,2018-01-02,30.0,已完成,3000.00,0.0,0.0,0.0,0.0
3,13953023,933376,空气滤清器,21.00,5,2018-01-02,NaN,NaN,2018-01-02,5.0,2018-01-02,5.0,已完成,105.00,0.0,0.0,0.0,0.0
4,13957873,QXC3643278243,机油滤清器,40.00,3,2018-01-01,NaN,NaN,2018-01-03,3.0,2018-01-03,3.0,已完成,120.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669910,21891811,QXC6346974943,NaN,140.00,1,2020-04-21,NaN,NaN,2020-04-21,1.0,2020-04-22,1.0,已完成,140.00,0.0,0.0,0.0,0.0
669911,21896946,QXC5220853570,轮胎,968.00,14,2020-04-24,NaN,NaN,2020-04-24,14.0,2020-04-25,14.0,已完成,13552.00,0.0,0.0,0.0,0.0
669912,21896946,QXC5220853570,轮胎,773.00,2,2020-04-24,NaN,NaN,2020-04-24,2.0,2020-04-25,2.0,已完成,1546.00,0.0,0.0,0.0,0.0
669913,21902829,JXS570571126,前刹车片,89.00,37,2020-04-21,NaN,NaN,2020-04-21,37.0,2020-04-22,37.0,已完成,3293.00,0.0,0.0,0.0,0.0


In [27]:
# 最早交易时间
tmp = order.groupby(['type_of_merchandize'])['order_time'].min().to_frame()
tmp.columns=[u'最早交易时间']
result_pinlei = tmp.reset_index()
result_pinlei.columns = [u'品类',u'最早交易时间']

# 最晚交易时间
tmp = order.groupby(['type_of_merchandize'])['order_time'].max().to_frame()
tmp.columns=[u'最近交易时间']
result_pinlei = pd.merge(result_pinlei, tmp, left_on=[u'品类'], right_index=True, how='left')

# 下单总次数
tmp = order.groupby(['type_of_merchandize'])['order_status'].count().to_frame()
tmp.columns=[u'下单总次数']
result_pinlei = pd.merge(result_pinlei, tmp, left_on=[u'品类'], right_index=True, how='left')

# 取消总次数
tmp = order[order['order_status']==u'其他'].groupby(['type_of_merchandize'])['order_status'].count().to_frame()
tmp.columns=[u'取消总次数']
result_pinlei = pd.merge(result_pinlei, tmp, left_on=[u'品类'], right_index=True, how='left')


# 完成总次数
tmp = order_finish.groupby(['type_of_merchandize'])['order_status'].count().to_frame()
tmp.columns=[u'完成总次数']
result_pinlei = pd.merge(result_pinlei, tmp, left_on=[u'品类'], right_index=True, how='left')

# 完成率
result_pinlei=result_pinlei.fillna(0)
result_pinlei[u'完成率'] = result_pinlei[u'完成总次数'] / result_pinlei[u'下单总次数'].apply(lambda x:float(x))
result_pinlei[u'完成率'] = result_pinlei[u'完成率'].apply(lambda x: round(x,4))

# 平均交易周期
result_pinlei[u'平均交易周期（天）'] = (result_pinlei[u'最近交易时间']-result_pinlei[u'最早交易时间']) / result_pinlei[u'下单总次数']
result_pinlei[u'平均交易周期（天）'] = result_pinlei[u'平均交易周期（天）'].apply(lambda x: round(x.total_seconds()/3600/24,1))

# 完成交易总金额
tmp = order_finish.groupby(['type_of_merchandize'])['pay_amt'].sum().to_frame()
tmp.columns=[u'完成交易总金额']
result_pinlei = pd.merge(result_pinlei, tmp, left_on=[u'品类'], right_index=True, how='left')

# 优惠次数
tmp = order_finish[order_finish['discount_amt']>0].groupby(['type_of_merchandize'])['discount_amt'].count().to_frame()
tmp.columns=[u'优惠次数']
result_pinlei = pd.merge(result_pinlei, tmp, left_on=[u'品类'], right_index=True, how='left')

# 优惠金额
tmp = order_finish[order_finish['discount_amt']>0].groupby(['type_of_merchandize'])['discount_amt'].sum().to_frame()
tmp.columns=['优惠总金额']
result_pinlei = pd.merge(result_pinlei, tmp, left_on=[u'品类'], right_index=True, how='left')


# 发货数量少于下单数量的次数
tmp = order_finish[order_finish[u'发货缺口']>0].groupby(['type_of_merchandize'])['order_ID'].count().to_frame()
tmp.columns=[u'发货缺口次数']
result_pinlei = pd.merge(result_pinlei, tmp, left_on=[u'品类'], right_index=True, how='left')

# 收货数量少于发货数量的次数
tmp = order_finish[order_finish[u'收货缺口']>0].groupby(['type_of_merchandize'])['order_ID'].count().to_frame()
tmp.columns=[u'收货缺口次数']
result_pinlei = pd.merge(result_pinlei, tmp, left_on=[u'品类'], right_index=True, how='left')

# 发货缺口金额
tmp = order_finish.groupby(['type_of_merchandize'])[u'发货缺口金额'].sum().to_frame()
tmp.columns=[u'发货缺口金额']
result_pinlei = pd.merge(result_pinlei, tmp, left_on=[u'品类'], right_index=True, how='left')

# 收货缺口金额
tmp = order_finish.groupby(['type_of_merchandize'])[u'收货缺口金额'].sum().to_frame()
tmp.columns=[u'收货缺口金额']
result_pinlei = pd.merge(result_pinlei, tmp, left_on=[u'品类'], right_index=True, how='left')

# 企业个数
tmp = order_finish.groupby(['type_of_merchandize','company_name'])['order_ID'].count().to_frame()
tmp = tmp.reset_index().groupby(['type_of_merchandize'])['company_name'].count().to_frame()
tmp.columns=[u'企业个数']
result_pinlei = pd.merge(result_pinlei, tmp, left_on=[u'品类'], right_index=True, how='left')

# 周交易密度
tmp = order_finish.groupby(['type_of_merchandize',order_finish.index.year, order_finish.index.week])['pay_amt'].count().to_frame().reset_index(level='type_of_merchandize')
tmp = tmp.groupby(['type_of_merchandize'])['pay_amt'].mean().to_frame()
tmp.columns=[u'周交易密度']
tmp = tmp.round(0)
result_pinlei = pd.merge(result_pinlei, tmp, left_on=[u'品类'], right_index=True, how='left')

# 周交易金额
tmp = order_finish.groupby(['type_of_merchandize',order_finish.index.year, order_finish.index.week])['pay_amt'].sum().to_frame().reset_index(level='type_of_merchandize')
tmp = tmp.groupby(['type_of_merchandize'])['pay_amt'].mean().to_frame()
tmp.columns=[u'周交易金额']
tmp = tmp.round(2)
result_pinlei = pd.merge(result_pinlei, tmp, left_on=[u'品类'], right_index=True, how='left')

# 月交易密度
tmp = order_finish.groupby(['type_of_merchandize',order_finish.index.year, order_finish.index.month])['pay_amt'].count().to_frame().reset_index(level='type_of_merchandize')
tmp = tmp.groupby(['type_of_merchandize'])['pay_amt'].mean().to_frame()
tmp.columns=[u'月交易密度']
tmp = tmp.round(0)
result_pinlei = pd.merge(result_pinlei, tmp, left_on=[u'品类'], right_index=True, how='left')

# 月交易金额
tmp = order_finish.groupby(['type_of_merchandize',order_finish.index.year, order_finish.index.month])['pay_amt'].sum().to_frame().reset_index(level='type_of_merchandize')
tmp = tmp.groupby(['type_of_merchandize'])['pay_amt'].mean().to_frame()
tmp.columns=[u'月交易金额']
tmp = tmp.round(2)
result_pinlei = pd.merge(result_pinlei, tmp, left_on=[u'品类'], right_index=True, how='left')



# fill NAN with 0.0
result_pinlei=result_pinlei.fillna(0)
result_pinlei

AttributeError: 'RangeIndex' object has no attribute 'year'

In [58]:
result_pinlei.loc['Row_sum'] = result_pinlei.apply(lambda x: col_desc(x))
today = datetime.date.today().strftime("%Y%m%d")
result_pinlei.to_excel('C:/Users/Vicky/Desktop/范式/中驰车福品类分析表.xlsx')

In [ ]:
#################################################
#可视化
#################################################

In [59]:
def company_vis(df, company_name, vis_col, figure_num):
    if vis_col == 'amt':
        industry = df.groupby([df.index.date,u'客户'])[u'应付金额'].sum().to_frame().reset_index()
        industry.columns = [u'下单时间',u'客户','stats']
        industry = industry.groupby([u'下单时间'])['stats'].mean().to_frame()
        
        company_df = df[df[u'客户']==company_name]
        company = company_df.groupby([company_df.index.date])[u'应付金额'].sum().to_frame()
        company.columns = ['stats']
        
    if vis_col == 'count':
        industry = df.groupby([df.index.date,u'客户'])[u'应付金额'].count().to_frame().reset_index()
        industry.columns = [u'下单时间',u'客户','stats']
        industry = industry.groupby([u'下单时间'])['stats'].mean().to_frame()

        company_df = df[df[u'客户']==company_name]
        company = company_df.groupby([company_df.index.date])[u'应付金额'].count().to_frame()
        company.columns = ['stats']
    
    p = pd.merge(industry, company, left_index=True,right_index=True, how='left')
    p=p.fillna(0)
    fig = plt.figure(figsize=(30,15))
    
    xweek=range(0,len(p.index))
    
    plt.plot(xweek, p['stats_x'].values, color='red',label='industry_mean')
    plt.plot(xweek, p['stats_y'].values,color='blue',label='company')
    
    plt.legend(loc='best', fontsize=25)
    plt.xticks(xweek,p.index,rotation=90)
    plt.yticks(fontsize=18)
    
    plt.xlabel(u'交易日', fontsize=25)
    
    if vis_col == 'amt':
        plt.ylabel(u'当日交易金额', fontsize=25)
        plt.title(u'{0}应付金额的日分布'.format(company_name), fontsize=25)
#         plt.savefig('./amt_day_{0}.png'.format(company_name))
    
    if vis_col == 'count':
        plt.ylabel(u'当日交易次数', fontsize=25)
        plt.title(u'{0}交易次数的日分布'.format(company_name), fontsize=25)
#         plt.savefig('./count_day_{0}.png'.format(company_name))



In [60]:
for i,com in enumerate(result[u'企业名称'].values[10:15]):
    company_vis(order, com, 'amt', i)
    company_vis(order, com, 'count', -i)

AttributeError: 'Int64Index' object has no attribute 'date'